In [1]:
import pyes
import elasticsearch
import json
import re
from operator import itemgetter

es_address='http://192.168.176.1:9200'
year = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
month = {	'Jan': {'value': 0, 'user': [],'total': []},  
			'Feb': {'value': 0, 'user': [],'total': []},  
			'Mar': {'value': 0, 'user': [],'total': []},  
			'Apr': {'value': 0, 'user': [],'total': []},  
			'May': {'value': 0, 'user': [],'total': []}, 
			'Jun': {'value': 0, 'user': [],'total': []}, 
			'Jul': {'value': 0, 'user': [],'total': []},  
			'Aug': {'value': 0, 'user': [],'total': []},  
			'Sep': {'value': 0, 'user': [],'total': []},  
			'Oct': {'value': 0, 'user': [],'total': []},  
			'Nov': {'value': 0, 'user': [],'total': []}, 
			'Dec': {'value': 0, 'user': [],'total': []} 
			}

conn = pyes.es.ES(es_address)
bq = pyes.query.BoolQuery() 
#q = pyes.query.MatchAllQuery()

cve = pyes.query.TermQuery(field="text", value="cve")
exp = pyes.query.TermQuery(field="text", value="exploit")

ESR = pyes.ESRange(field="created_at", 
				   from_value="Mon Sep 01 00:00:00 +0000 2014", to_value="Mon Sep 01 00:00:00 +0000 2015", 
                   include_lower=True ,include_upper=False)
rq = pyes.query.RangeQuery(qrange=ESR)

bq.add_must(rq)
bq.add_should(cve)
bq.add_should(exp)
#bq.add_should(tq)

result = conn.search(query=bq) 
#json.dumps(result[0],indent=2)

user = []
url = {'T': 0, 'F': 0}
for i in result:  #a,b,c,d,e
    uid = i['uid']
    m = i['created_at'].split(' ')[1]
    text = i['text']
    if uid not in user:
        user.append(uid)
    if uid not in month[m]['user']:
    	month[m]['user'].append(uid)

    month[m]['total'].append(uid)
    month[m]['value'] +=1 ;

    if len(re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)) != 0 :
    	url['T'] += 1
    else :
    	url['F'] += 1

print 'tweets',len(result)
print 'twitter',len(user)
for m in year: # f
	print '==================',m,'==================='
	value = month[m]['value']
	p = len(month[m]['user'])
	count = {}
	user = month[m]['user']
	total = month[m]['total']
	of = len(month[m]['user'])/4

	for n in user:
		count[n] = 0
	for n in total:
		count[n] +=1
	data = sorted(count.iteritems(), key=lambda (k,v): (v,k), reverse = True)
	month[m]['avg'] = data
	if len(data) != 0 :
		print 'Most :', data[0][0], data[0][1]
		print '3 quartile :', data[of][0], data[of][1]
		print 'avg :', float(value)/float(p)
		print '1 quartile :', data[-of][0], data[-of][1]
		print 'Least :', data[-1][0], data[-1][1]

		print 'tweet :', value, '\t',
		print 'twitter :', p, '\t'
	print ''
print url



tweets 3416
twitter 279
================== Jan ===================
Most : 43130563 8
3 quartile : 13275122 4
avg : 2.5
1 quartile : 18783289 1
Least : 48443 1
tweet : 105 	twitter : 42 	

================== Feb ===================
Most : 405658492 27
3 quartile : 13275122 4
avg : 3.31707317073
1 quartile : 16730420 1
Least : 3247471 1
tweet : 136 	twitter : 41 	

================== Mar ===================
Most : 405658492 86
3 quartile : 578959213197537280 3
avg : 4.61333333333
1 quartile : 297856522 1
Least : 11443182 1
tweet : 346 	twitter : 75 	

================== Apr ===================
Most : 588352148389269505 56
3 quartile : 593218657192390657 6
avg : 6.6393442623
1 quartile : 390189754 1
Least : 810641 1
tweet : 405 	twitter : 61 	

================== May ===================
Most : 603329449388429312 36
3 quartile : 598021356681310208 6
avg : 5.21739130435
1 quartile : 297856522 1
Least : 1102 1
tweet : 360 	twitter : 69 	

================== Jun ===================
Most : 608

In [2]:
import numpy
import pandas
from bokeh import charts, plotting, models
from bokeh.charts import Donut
from bokeh.sampledata.autompg import autompg as df

plotting.output_notebook()

#blue, green = bokeh.palettes.Paired4


datalist = [
    {'month': '01_Sep'},
    {'month': '02_Oct'},
    {'month': '03_Nov'},
    {'month': '04_Dec'},
    {'month': '05_Jan'},
    {'month': '06_Feb'},
    {'month': '07_Mar'},
    {'month': '08_Apr'},
    {'month': '09_May'},
    {'month': '10_Jun'},
    {'month': '11_Jul'},
    {'month': '12_Aug'},
]

for i in datalist:
    for j in year:
        if j in i['month']:
            i['tweets'] = month[j]['value']
            i['twitter'] = len(month[j]['user'])

data = pandas.DataFrame(datalist)


bar = charts.Bar(data=data, values='tweets', label='month', title='tweets',
               color='blue', width=800, height=700)
plotting.show(bar)

#scatter = charts.Scatter(data=data, x='month', y='rain',
#                         color='blue', width=600, height=300)
#plotting.show(scatter)

Loading BokehJS ...

In [3]:
bar = charts.Bar(data=data, values='twitter', label='month', title='twitter',
               color='blue', width=800, height=700)
plotting.show(bar)

In [4]:
from bokeh.charts import Donut, BoxPlot
import pandas as pd

# utilize utility to make it easy to get json/dict data converted to a dataframe


# filter by countries with at least one medal and sort by total medals

data_url = []
for k,v in url.items():
    data_url.append({'url':k,'value':v})

data_url = pandas.DataFrame(data_url)

# original example
d = Donut(data_url, label=['url'], values='value',
          text_font_size='16pt',hover_text='tweets', title='URL',
          width=600, height=600)


plotting.show(d)

In [185]:

avg = []
for m in year:
    for t in month[m]['avg']:
        avg.append({'month':m,'tweets':t[1]})
avg = pandas.DataFrame(avg)

p = BoxPlot(avg, values='tweets', label='month', outliers=False,
            title="MPG Summary (grouped by CYL, no outliers)")

output_notebook()

show(p)

Loading BokehJS ...